In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
from neomodel import FloatProperty, IntegerProperty, StringProperty, StructuredNode, StructuredRel


class Student(StructuredNode):
    name = StringProperty(unique_index=True)
    age = IntegerProperty()


class SalaryRecord(StructuredNode):
    employee_id = StringProperty(unique_index=True)
    amount = FloatProperty()

In [ ]:
from neomodel import StructuredNode, config, db


class Neo4jNeomodelRepository:
    def __init__(self, uri: str, username: str, password: str):
        auth_uri = uri.replace('bolt://', f'bolt://{username}:{password}@')
        config.DATABASE_URL = auth_uri

    async def create_node(self, model_class: type[StructuredNode], properties: dict):
        node = model_class(**properties).save()

        return node

    async def find_nodes(self, model_class: type[StructuredNode], match_properties: dict = None):
        if match_properties:
            qs = model_class.nodes.filter(**match_properties)
        else:
            qs = model_class.nodes

        return list(qs)

    async def delete_nodes(self, model_class: type[StructuredNode], match_properties: dict):
        qs = model_class.nodes.filter(**match_properties)
        count = qs.count()
        qs.delete()

        return count

    async def update_node(
        self, model_class: type[StructuredNode], match_properties: dict, update_properties: dict
    ):
        node = model_class.nodes.get_or_none(**match_properties)
        if node:
            for key, value in update_properties.items():
                setattr(node, key, value)
            node.save()
            return node

        return None

    async def create_relationship(
        self,
        from_model: type[StructuredNode],
        from_match: dict,
        rel_name: str,
        to_model: type[StructuredNode],
        to_match: dict,
        rel_props: dict = None,
    ):
        src = from_model.nodes.get_or_none(**from_match)
        if not src:
            src = from_model(**from_match).save()

        dst = to_model.nodes.get_or_none(**to_match)
        if not dst:
            dst = to_model(**to_match).save()

        rel_manager = getattr(src, rel_name, None)
        if not rel_manager:
            raise AttributeError(f"Relationship '{rel_name}' not defined on {from_model.__name__}")

        if rel_props:
            rel = rel_manager.connect(dst, **rel_props)
        else:
            rel = rel_manager.connect(dst)
        return rel

    async def delete_relationships(
        self,
        from_model: type[StructuredNode],
        from_match: dict,
        rel_name: str,
        to_model: type[StructuredNode],
        to_match: dict,
    ):
        src = from_model.nodes.get_or_none(**from_match)
        dst = to_model.nodes.get_or_none(**to_match)
        if not src or not dst:
            return 0

        rel_manager = getattr(src, rel_name, None)
        if not rel_manager:
            raise AttributeError(f"Relationship '{rel_name}' not defined on {from_model.__name__}")

        rels = rel_manager.match(dst)
        count = 0
        for r in rels:
            r.delete()
            count += 1

        return count

In [ ]:
class StudentGraphRepository(Neo4jNeomodelRepository):
    def __init__(self, uri: str, username: str, password: str):
        super().__init__(uri, username, password)

    async def create_student(self, name: str, age: int):
        return await self.create_node(Student, {'name': name, 'age': age})

    async def find_students(self, filters: dict = None):
        return await self.find_nodes(Student, filters)

    async def delete_students(self, filters: dict):
        return await self.delete_nodes(Student, filters)

    async def update_student(self, match_properties: dict, update_properties: dict):
        return await self.update_node(Student, match_properties, update_properties)

In [ ]:
class SalaryGraphRepository(Neo4jNeomodelRepository):
    def __init__(self, uri: str, username: str, password: str):
        super().__init__(uri, username, password)

    async def create_salary(self, employee_id: str, amount: float):
        return await self.create_node(SalaryRecord, {'employee_id': employee_id, 'amount': amount})

    async def find_salary_records(self, filters: dict = None):
        return await self.find_nodes(SalaryRecord, filters)

    async def delete_salary_records(self, filters: dict):
        return await self.delete_nodes(SalaryRecord, filters)

    async def update_salary_record(self, match_properties: dict, update_properties: dict):
        return await self.update_node(SalaryRecord, match_properties, update_properties)

In [ ]:
user = 'neo4j'
pw = 'password'
uri = f'bolt://neo4j:7687'


student_repo = StudentGraphRepository(uri, user, pw)
await student_repo.create_student('Alice', 20)
students = await student_repo.find_students({'age': 20})
print('Students aged 20:', [s.name for s in students])

salary_repo = SalaryGraphRepository(uri, user, pw)
await salary_repo.create_salary('emp123', 55000.0)
salaries = await salary_repo.find_salary_records({'employee_id': 'emp123'})
print('Salary for emp123:', [r.amount for r in salaries])

Students aged 20: ['Alice', 'Alice']
Salary for emp123: [55000.0]
